<a href="https://colab.research.google.com/github/Aayushmatkar/DataScienceApplications/blob/main/final%20GROUP12_ASSIGNMENT_1(classification).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#lib imports 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
#MODEL IMPORTS
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
#VALIDATION METRICS
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

import matplotlib.pyplot as plt

#DATA PREPARATION AND CLEANING 

In [ ]:
import nltk
from nltk.corpus import gutenberg
import re
import os,random
import pandas as pd
nltk.download('punkt')
nltk.download('gutenberg')

def create_partitions(text, book_name, author):
    partitions = []
    words = nltk.word_tokenize(text)
    for i in range(0, len(words), 100):
        partitions.append((words[i:i+100], book_name, author))
    return partitions

#The books selected below are from the gutenberg module and are of the same genre 'novel'
book_names=['melville-moby_dick.txt','edgeworth-parents.txt','chesterton-thursday.txt','austen-emma.txt','milton-paradise.txt']

def random_sentences_with_book_name():
  result=pd.DataFrame(columns=['Text', 'Book Name','Author'])
  for i in book_names:
    # Splitting the book names to remove their 'txt' extension
    label_name=i.split('.')[0]

    # Getting the book names from Gutenberg and reading them
    book = gutenberg.raw(i)

    #Getting the book names and their respective authors
    lines = book.splitlines()

    #The first line consists information about the above comment
    line1=lines[0]
    
    #Removing the year from the line
    pattern = r'\D+'

    digits_removed = re.findall(pattern, line1)

    #Getting the author name
    author = re.search(r'\sby\s([\w\s\.]+)[\s\d{4}]+', digits_removed[0]).group(1)

    #Getting the book name
    book_name = re.search(r'\[([\w\s\']+)[\,]?\sby', line1).group(1)

    # Create partitions of 100 words each
    partitions = create_partitions(book, book_name, author)

    # Create 200 random samples of the partitions
    samples = random.sample(partitions, 200)

    # Using regular expressions to clean the data
    for i in range(len(samples)):
        samples[i] = (re.sub(r'[^\w\s]','', ' '.join(samples[i][0])), samples[i][1], samples[i][2])

    # Serialize the data using Pandas
    df = pd.DataFrame(samples, columns=['Text', 'Book Name','Author'])

    #Appending the random sentences of a book to the final dataframe 
    result=pd.concat([result,df])
  result.to_csv('book_partitions.csv', index=False)

  print("Data saved to book_partitions.csv.")

In [ ]:
random_sentences_with_book_name()
data=pd.read_csv('book_partitions.csv')
data['Author'].value_counts(),data['Author'].unique()
data['Book Name'].value_counts(),data['Book Name'].unique()

In [ ]:
data['label']='a'
data['index']=0
data['label'][200:400]='b'
data['index'][200:400]=1
data['label'][400:600]='c'
data['index'][400:600]=2
data['label'][600:800]='d'
data['index'][600:800]=3
data['label'][800:1000]='e'
data['index'][800:1000]=4

In [ ]:
data


In [ ]:
# plot word cloud for the most frequent 20 words in each book  
import wordcloud
import matplotlib.pyplot as plt
import plotly.express as px


for label in data['label'].unique():
  books = data[data["label"]==label]["Text"]


# Create word cloud
  wc = wordcloud.WordCloud(background_color=None, max_words=20, 
                          max_font_size=100)
  wc = wc.generate(str(books))

# Convert word cloud to dataframe
  words = list(wc.words_.keys())
  freq = [wc.words_[word] for word in words]
  data10 = {'words': words, 'freq': freq}
  df10 = pd.DataFrame(data10)

  # Create interactive word cloud
  fig = px.scatter(df10, x='words', y='freq', text='words', size='freq', color='freq')
  fig.update_layout(title='Interactive Word Cloud',
                    xaxis_title='Words',
                    yaxis_title='Frequency')
  fig.show()

#BAG OF WORDS 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt


# Convert the text data into a tf-idf representation
vectorizer = CountVectorizer()

bag_of_words = vectorizer.fit_transform(data['Text'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(bag_of_words, data['Author'], test_size=0.2)


# K-NEAREST NEIGHBOU train and evaluate 
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_test)


knn_accuracy = accuracy_score(y_test, knn_pred)
knn_precision = precision_score(y_test, knn_pred,average='macro')
knn_recall = recall_score(y_test, knn_pred,average='macro')
knn_fscore = f1_score (y_test, knn_pred,average='macro')

print('knn_accuracy with Bag of Words = ',knn_accuracy)
print('knn_precision with Bag of Words = ',knn_precision)
print('knn_recall with Bag of Words = ',knn_recall)
print('knn_Fscore with Bag of Words = ',knn_fscore)
print("----------------------------------------------------------------")


#DECISION TREE 
# Train and evaluate the Decision Tree model
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
dt_pred = dt.predict(X_test)



dt_accuracy = accuracy_score(y_test, dt_pred)
dt_precision = precision_score(y_test, dt_pred,average='macro')
dt_recall = recall_score(y_test, dt_pred,average='macro')
dt_fscore = f1_score (y_test, dt_pred,average='macro')

print('dt_accuracy with Bag of Words  = ',dt_accuracy)
print('dt_precision with Bag of Words = ',dt_precision)
print('dt_recall with Bag of Words = ',dt_recall)
print('dt_Fscore with Bag of Words = ',dt_fscore)
print("----------------------------------------------------------------")



# Train and evaluate the SVM model
svm = SVC()
svm.fit(X_train, y_train)
svm_pred = svm.predict(X_test)


svm_accuracy = accuracy_score(y_test, svm_pred)
svm_precision = precision_score(y_test, svm_pred,average='macro')
svm_recall = recall_score(y_test, svm_pred,average='macro')
svm_fscore = f1_score (y_test, svm_pred,average='macro')


print('svm_accuracy with Bag of Words = ',svm_accuracy)
print('svm_precision with Bag of Words = ',svm_precision)
print('svm_recall with Bag of Words = ',svm_recall)
print('svm_Fscore with Bag of Words = ',svm_fscore)
print("----------------------------------------------------------------")

# Train and evaluate the XGBoost model
xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_train, y_train)
xgb_pred = xgb_model.predict(X_test)

xgb_accuracy = accuracy_score(y_test, xgb_pred)
xgb_precision = precision_score(y_test, xgb_pred,average='macro')
xgb_recall = recall_score(y_test, xgb_pred,average='macro')
xgb_fscore = f1_score (y_test, xgb_pred,average='macro')


print('xgb_accuracy with Bag of Words = ',xgb_accuracy)
print('xgb_precision with Bag of Words = ',xgb_precision)
print('xgb_recall with Bag of Words = ',xgb_recall)
print('xgb_Fscore with Bag of Words = ',xgb_fscore)
print("----------------------------------------------------------------")
#++++++++++++++++++++++++++++++++++++++++++++++++


# Plot the accuracy comparison of the models
models = ['KNN', 'SVM','DT', 'XGBoost']
accuracies = [knn_accuracy, svm_accuracy,dt_accuracy, xgb_accuracy]
plt.bar(models, accuracies)
plt.xlabel('Models')
plt.ylabel('Accuracy')
plt.title('Accuracy comparison of models')
plt.show()

# Plot the Precision comparison of the models
models = ['KNN', 'SVM','DT', 'XGBoost']
precisions = [knn_precision, svm_precision,dt_precision, xgb_precision]
plt.bar(models, precisions)
plt.xlabel('Models')
plt.ylabel('precision')
plt.title('Precision comparison of models')
plt.show()

# Plot the Recall comparison of the models
models = ['KNN', 'SVM','DT', 'XGBoost']
recalls = [knn_recall, svm_recall,dt_recall, xgb_recall]
plt.bar(models, recalls)
plt.xlabel('Models')
plt.ylabel('Recall')
plt.title('Recall comparison of models')
plt.show()

# Plot the Fscore comparison of the models
models = ['KNN', 'SVM','DT', 'XGBoost']
fscores = [knn_fscore, svm_fscore,dt_fscore, xgb_fscore]
plt.bar(models, fscores)
plt.xlabel('Models')
plt.ylabel('Fscore')
plt.title('Fscore comparison of models')
plt.show()

# CONFUSION MATRIX AND ERROR ANALYSIS FOR BAG OF WORDS 

In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix

#SVM ERROR ANALYSIS AND CONFUSION MATRIX 

In [ ]:
temp=[]
for i in y_test.index:
  temp.append(data['Text'].iloc[i])
df=pd.DataFrame(temp,columns=['Text'])
svm_pred = svm.predict(X_test)
df['actual_data']=y_test.to_numpy()
df['predicted_data']=svm_pred
df

In [ ]:
df['Count']=df['predicted_data'] == df['actual_data']
count=0
for i in df['Count']:
  if i ==True:
    count=count+1
df.drop(labels=['Count'],axis=1,inplace=True)
print("The number of wrong predictions made by the SVM model with Bag Of Words are:",200-count)

conf_matrix = confusion_matrix(y_test.to_numpy(), svm_pred)
conf_matrix_df = pd.DataFrame(conf_matrix, index=['Herman', 'Maria','Chesterton','Jane','Milton'], columns=['Herman', 'Maria','Chesterton','Jane','Milton'])
print('\nThe confusion matrix for the SVM model is \n\n',conf_matrix_df)
print("--------------------------------------------------------------------------------------------")

In [ ]:
import seaborn as sns


conf_matrix = confusion_matrix(y_test.to_numpy(), svm_pred)
conf_matrix_df = pd.DataFrame(conf_matrix, index=['Herman', 'Maria','Chesterton','Jane','Milton'], columns=['Herman', 'Maria','Chesterton','Jane','Milton'])
sns.heatmap(conf_matrix_df, annot=True, fmt='d')
plt.title("Confusion Matrix for SVM Model")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

#KNN ERROR AND CONFUSION MATRIX

In [ ]:
temp=[]
for i in y_test.index:
  temp.append(data['Text'].iloc[i])
df=pd.DataFrame(temp,columns=['Text'])
knn_pred = knn.predict(X_test)
df['actual_data']=y_test.to_numpy()
df['predicted_data']=knn_pred
df

In [ ]:
df['Count']=df['predicted_data'] == df['actual_data']
count=0
for i in df['Count']:
  if i ==True:
    count=count+1
df.drop(labels=['Count'],axis=1,inplace=True)
print("The number of wrong predictions made by the KNN model with Bag Of Words are:",200-count)

conf_matrix = confusion_matrix(y_test.to_numpy(), knn_pred)
conf_matrix_df = pd.DataFrame(conf_matrix, index=['Herman', 'Maria','Chesterton','Jane','Milton'], columns=['Herman', 'Maria','Chesterton','Jane','Milton'])
print('\nThe confusion matrix for the KNN model is \n\n',conf_matrix_df)
print("--------------------------------------------------------------------------------------------")



In [ ]:
conf_matrix = confusion_matrix(y_test.to_numpy(), knn_pred)
conf_matrix_df = pd.DataFrame(conf_matrix, index=['Herman', 'Maria','Chesterton','Jane','Milton'], columns=['Herman', 'Maria','Chesterton','Jane','Milton'])
sns.heatmap(conf_matrix_df, annot=True, fmt='d')
plt.title("Confusion Matrix for KNN Model")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

#Decision Tree ERROR AND CONFUSION MATRIX

In [ ]:
temp=[]
for i in y_test.index:
  temp.append(data['Text'].iloc[i])
df=pd.DataFrame(temp,columns=['Text'])
dt_pred = dt.predict(X_test)
df['actual_data']=y_test.to_numpy()
df['predicted_data']=dt_pred
df

In [ ]:
df['Count']=df['predicted_data'] == df['actual_data']
count=0
for i in df['Count']:
  if i ==True:
    count=count+1
df.drop(labels=['Count'],axis=1,inplace=True)
print("The number of wrong predictions made by the decision tree model with Bag Of Words are:",200-count)

conf_matrix = confusion_matrix(y_test.to_numpy(), dt_pred)
conf_matrix_df = pd.DataFrame(conf_matrix, index=['Herman', 'Maria','Chesterton','Jane','Milton'], columns=['Herman', 'Maria','Chesterton','Jane','Milton'])
print('\nThe confusion matrix for the Decision Tree model is \n\n',conf_matrix_df)
print("--------------------------------------------------------------------------------------------")


In [ ]:
conf_matrix = confusion_matrix(y_test.to_numpy(), dt_pred)
conf_matrix_df = pd.DataFrame(conf_matrix, index=['Herman', 'Maria','Chesterton','Jane','Milton'], columns=['Herman', 'Maria','Chesterton','Jane','Milton'])
sns.heatmap(conf_matrix_df, annot=True, fmt='d')
plt.title("Confusion Matrix for Decision Tree Model")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

#XGBoost ERROR AND CONFUSION MATRIX 

In [ ]:
temp=[]
for i in y_test.index:
  temp.append(data['Text'].iloc[i])
df=pd.DataFrame(temp,columns=['Text'])
xgb_pred = xgb_model.predict(X_test)
df['actual_data']=y_test.to_numpy()
df['predicted_data']=xgb_pred
df

In [ ]:
df['Count']=df['predicted_data'] == df['actual_data']
count=0
for i in df['Count']:
  if i ==True:
    count=count+1
df.drop(labels=['Count'],axis=1,inplace=True)
print("The number of wrong predictions made by the XGBoost model with Bag Of Words are:",200-count)

conf_matrix = confusion_matrix(y_test.to_numpy(), xgb_pred)
conf_matrix_df = pd.DataFrame(conf_matrix, index=['Herman', 'Maria','Chesterton','Jane','Milton'], columns=['Herman', 'Maria','Chesterton','Jane','Milton'])
print('\nThe confusion matrix for the XGBoost model is \n\n',conf_matrix_df)
print("--------------------------------------------------------------------------------------------")

In [ ]:
conf_matrix = confusion_matrix(y_test.to_numpy(), xgb_pred)
conf_matrix_df = pd.DataFrame(conf_matrix, index=['Herman', 'Maria','Chesterton','Jane','Milton'], columns=['Herman', 'Maria','Chesterton','Jane','Milton'])
sns.heatmap(conf_matrix_df, annot=True, fmt='d')
plt.title("Confusion Matrix for XGBoost Model")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

#10 fold cross validation

In [ ]:
import numpy as np
from sklearn.model_selection import KFold

def ten_fold_cross_validation(model, X, y):
    kf = KFold(n_splits=10)
    accuracy = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        model.fit(X_train, y_train)
        accuracy.append(model.score(X_test, y_test))
    
    return np.array(accuracy)


In [ ]:
print("10 fold validation for SVM \n")
accuracies=ten_fold_cross_validation(svm,bag_of_words,data['Author'])
print(accuracies)
print("Bias and Variability for SVM model is\n")
bias = 1-np.mean(accuracies)
variability = np.std(accuracies)

print("Bias: {:.3f}".format(bias))
print("Variability: {:.3f}".format(variability))

In [ ]:
print("10 fold validation for KNN \n")
accuracies=ten_fold_cross_validation(knn,bag_of_words,data['Author'])
print(accuracies)
print("Bias and Variability for KNN model is\n")
bias = 1-np.mean(accuracies)
variability = np.std(accuracies)

print("Bias: {:.3f}".format(bias))
print("Variability: {:.3f}".format(variability))

In [ ]:
print("10 fold validation for Decision Tree \n")
accuracies=ten_fold_cross_validation(dt,bag_of_words,data['Author'])
print(accuracies)
print("Bias and Variability for Decision Tree model is\n")
bias = 1-np.mean(accuracies)
variability = np.std(accuracies)

print("Bias: {:.3f}".format(bias))
print("Variability: {:.3f}".format(variability))

In [ ]:
print("10 fold validation for XGB \n")
accuracies=ten_fold_cross_validation(xgb_model,bag_of_words,data['Author'])
print(accuracies)
print("Bias and Variability for XGB model is\n")
bias = 1-np.mean(accuracies)
variability = np.std(accuracies)

print("Bias: {:.3f}".format(bias))
print("Variability: {:.3f}".format(variability))

# TF-IDF 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# Read the csv file
#df = pd.read_csv('book_partitions.csv')

# Convert the text data into a tf-idf representation
vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(data['Text'].values)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(tfidf, data['Author'], test_size=0.2, random_state=42)

# K-NEAREST NEIGHBOUR 
# Train and evaluate the KNN model
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_test)


knn_accuracy = accuracy_score(y_test, knn_pred)
knn_precision = precision_score(y_test, knn_pred,average='macro')
knn_recall = recall_score(y_test, knn_pred,average='macro')
knn_fscore = f1_score (y_test, knn_pred,average='macro')

print('knn_accuracy with TF-IDF = ',knn_accuracy)
print('knn_precision with TF-IDF = ',knn_precision)
print('knn_recall with TF-IDF = ',knn_recall)
print('knn_Fscore with TF-IDF = ',knn_fscore)
print("----------------------------------------------------------------")


#DECISION TREE 
# Train and evaluate the Decision Tree model
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
dt_pred = dt.predict(X_test)



dt_accuracy = accuracy_score(y_test, dt_pred)
dt_precision = precision_score(y_test, dt_pred,average='macro')
dt_recall = recall_score(y_test, dt_pred,average='macro')
dt_fscore = f1_score (y_test, dt_pred,average='macro')

print('dt_accuracy with TF-IDF  = ',dt_accuracy)
print('dt_precision with TF-IDF = ',dt_precision)
print('dt_recall with TF-IDF = ',dt_recall)
print('dt_Fscore with TF-IDF = ',dt_fscore)
print("----------------------------------------------------------------")



# Train and evaluate the SVM model
svm = SVC()
svm.fit(X_train, y_train)
svm_pred = svm.predict(X_test)


svm_accuracy = accuracy_score(y_test, svm_pred)
svm_precision = precision_score(y_test, svm_pred,average='macro')
svm_recall = recall_score(y_test, svm_pred,average='macro')
svm_fscore = f1_score (y_test, svm_pred,average='macro')


print('svm_accuracy with TF-IDF = ',svm_accuracy)
print('svm_precision with TF-IDF = ',svm_precision)
print('svm_recall with TF-IDF = ',svm_recall)
print('svm_Fscore with TF-IDF = ',svm_fscore)
print("----------------------------------------------------------------")

# Train and evaluate the XGBoost model
xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_train, y_train)
xgb_pred = xgb_model.predict(X_test)

xgb_accuracy = accuracy_score(y_test, xgb_pred)
xgb_precision = precision_score(y_test, xgb_pred,average='macro')
xgb_recall = recall_score(y_test, xgb_pred,average='macro')
xgb_fscore = f1_score (y_test, xgb_pred,average='macro')


print('xgb_accuracy with TF-IDF = ',xgb_accuracy)
print('xgb_precision with TF-IDF = ',xgb_precision)
print('xgb_recall with TF-IDF = ',xgb_recall)
print('xgb_Fscore with TF-IDF = ',xgb_fscore)
print("----------------------------------------------------------------")
#++++++++++++++++++++++++++++++++++++++++++++++++


# Plot the accuracy comparison of the models
models = ['KNN', 'SVM','DT', 'XGBoost']
accuracies = [knn_accuracy, svm_accuracy,dt_accuracy, xgb_accuracy]
plt.bar(models, accuracies)
plt.xlabel('Models')
plt.ylabel('Accuracy')
plt.title('Accuracy comparison of models')
plt.show()

# Plot the Precision comparison of the models
models = ['KNN', 'SVM','DT', 'XGBoost']
precisions = [knn_precision, svm_precision,dt_precision, xgb_precision]
plt.bar(models, precisions)
plt.xlabel('Models')
plt.ylabel('precision')
plt.title('Precision comparison of models')
plt.show()

# Plot the Recall comparison of the models
models = ['KNN', 'SVM','DT', 'XGBoost']
recalls = [knn_recall, svm_recall,dt_recall, xgb_recall]
plt.bar(models, recalls)
plt.xlabel('Models')
plt.ylabel('Recall')
plt.title('Recall comparison of models')
plt.show()

# Plot the Fscore comparison of the models
models = ['KNN', 'SVM','DT', 'XGBoost']
fscores = [knn_fscore, svm_fscore,dt_fscore, xgb_fscore]
plt.bar(models, fscores)
plt.xlabel('Models')
plt.ylabel('Fscore')
plt.title('Fscore comparison of models')
plt.show()

#CONFUSION MATRIX AND ERROR ANALYSIS FOR TF-IDF

In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix

# SVM ERROR ANALYSIS AND CONFUSION MATRIX

In [ ]:
temp=[]
for i in y_test.index:
  temp.append(data['Text'].iloc[i])
df=pd.DataFrame(temp,columns=['Text'])
svm_pred = svm.predict(X_test)
df['actual_data']=y_test.to_numpy()
df['predicted_data']=svm_pred
df

In [ ]:
df['Count']=df['predicted_data'] == df['actual_data']
count=0
for i in df['Count']:
  if i ==True:
    count=count+1
df.drop(labels=['Count'],axis=1,inplace=True)
print("The number of wrong predictions made by the SVM model with TF-IDF are:",200-count)

conf_matrix = confusion_matrix(y_test.to_numpy(), svm_pred)
conf_matrix_df = pd.DataFrame(conf_matrix, index=['Herman', 'Maria','Chesterton','Jane','Milton'], columns=['Herman', 'Maria','Chesterton','Jane','Milton'])
print('\nThe confusion matrix for the SVM model is \n\n',conf_matrix_df)
print("--------------------------------------------------------------------------------------------")

In [ ]:
sns.heatmap(conf_matrix_df, annot=True, fmt='d')
plt.title("Confusion Matrix for SVM Model")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

# KNN ERROR AND CONFUSION MATRIX

In [ ]:
temp=[]
for i in y_test.index:
  temp.append(data['Text'].iloc[i])
df=pd.DataFrame(temp,columns=['Text'])
knn_pred = knn.predict(X_test)
df['actual_data']=y_test.to_numpy()
df['predicted_data']=knn_pred
df

In [ ]:
df['Count']=df['predicted_data'] == df['actual_data']
count=0
for i in df['Count']:
  if i ==True:
    count=count+1
df.drop(labels=['Count'],axis=1,inplace=True)
print("The number of wrong predictions made by the KNN model with TF-IDF are:",200-count)

conf_matrix = confusion_matrix(y_test.to_numpy(), knn_pred)
conf_matrix_df = pd.DataFrame(conf_matrix, index=['Herman', 'Maria','Chesterton','Jane','Milton'], columns=['Herman', 'Maria','Chesterton','Jane','Milton'])
print('\nThe confusion matrix for the KNN model is \n\n',conf_matrix_df)
print("--------------------------------------------------------------------------------------------")


In [ ]:
sns.heatmap(conf_matrix_df, annot=True, fmt='d')
plt.title("Confusion Matrix for KNN Model")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

#Decision Tree ERROR AND CONFUSION MATRIX

In [ ]:
temp=[]
for i in y_test.index:
  temp.append(data['Text'].iloc[i])
df=pd.DataFrame(temp,columns=['Text'])
dt_pred = dt.predict(X_test)
df['actual_data']=y_test.to_numpy()
df['predicted_data']=dt_pred
df

In [ ]:
df['Count']=df['predicted_data'] == df['actual_data']
count=0
for i in df['Count']:
  if i ==True:
    count=count+1
df.drop(labels=['Count'],axis=1,inplace=True)
print("The number of wrong predictions made by the decision tree model with TF-IDF are:",200-count)

conf_matrix = confusion_matrix(y_test.to_numpy(), dt_pred)
conf_matrix_df = pd.DataFrame(conf_matrix, index=['Herman', 'Maria','Chesterton','Jane','Milton'], columns=['Herman', 'Maria','Chesterton','Jane','Milton'])
print('\nThe confusion matrix for the Decision Tree model is \n\n',conf_matrix_df)
print("--------------------------------------------------------------------------------------------")

In [ ]:
sns.heatmap(conf_matrix_df, annot=True, fmt='d')
plt.title("Confusion Matrix for Decision Tree Model")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

# XGBoost ERROR AND CONFUSION MATRIX

In [ ]:
temp=[]
for i in y_test.index:
  temp.append(data['Text'].iloc[i])
df=pd.DataFrame(temp,columns=['Text'])
xgb_pred = xgb_model.predict(X_test)
df['actual_data']=y_test.to_numpy()
df['predicted_data']=xgb_pred
df

In [ ]:
df['Count']=df['predicted_data'] == df['actual_data']
count=0
for i in df['Count']:
  if i ==True:
    count=count+1
df.drop(labels=['Count'],axis=1,inplace=True)
print("The number of wrong predictions made by the XGBoost model with TF-IDF are:",200-count)

conf_matrix = confusion_matrix(y_test.to_numpy(), xgb_pred)
conf_matrix_df = pd.DataFrame(conf_matrix, index=['Herman', 'Maria','Chesterton','Jane','Milton'], columns=['Herman', 'Maria','Chesterton','Jane','Milton'])
print('\nThe confusion matrix for the XGBoost model is \n\n',conf_matrix_df)
print("--------------------------------------------------------------------------------------------")

In [ ]:
sns.heatmap(conf_matrix_df, annot=True, fmt='d')
plt.title("Confusion Matrix for XGBoost Model")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

# 10 Fold Cross Validation

In [ ]:
print("10 fold validation for SVM \n")
accuracies=ten_fold_cross_validation(svm,tfidf,data['Author'])
print(accuracies)
print("Bias and Variability for SVM model is\n")
bias = 1-np.mean(accuracies)
variability = np.std(accuracies)

print("Bias: {:.3f}".format(bias))
print("Variability: {:.3f}".format(variability))

In [ ]:
print("10 fold validation for KNN \n")
accuracies=ten_fold_cross_validation(knn,tfidf,data['Author'])
print(accuracies)
print("Bias and Variability for KNN model is\n")
bias = 1-np.mean(accuracies)
variability = np.std(accuracies)

print("Bias: {:.3f}".format(bias))
print("Variability: {:.3f}".format(variability))

In [ ]:
print("10 fold validation for Decision Tree \n")
accuracies=ten_fold_cross_validation(dt,tfidf,data['Author'])
print(accuracies)
print("Bias and Variability for Decision Tree model is\n")
bias = 1-np.mean(accuracies)
variability = np.std(accuracies)

print("Bias: {:.3f}".format(bias))
print("Variability: {:.3f}".format(variability))

In [ ]:
print("10 fold validation for XGB \n")
accuracies=ten_fold_cross_validation(xgb_model,tfidf,data['Author'])
print(accuracies)
print("Bias and Variability for XGB model is\n")
bias = 1-np.mean(accuracies)
variability = np.std(accuracies)

print("Bias: {:.3f}".format(bias))
print("Variability: {:.3f}".format(variability))

# N-GRAM


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# Read the csv file
#df = pd.read_csv('book_partitions3.csv')

# Convert the text data into a n-gram representation
vectorizer = CountVectorizer(ngram_range=(1, 2))
ngrams = vectorizer.fit_transform(data['Text'].values)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(ngrams, data['Author'], test_size=0.2, random_state=42)

# K-NEAREST NEIGHBOUR 
# Train and evaluate the KNN model
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_test)


knn_accuracy = accuracy_score(y_test, knn_pred)
knn_precision = precision_score(y_test, knn_pred,average='macro')
knn_recall = recall_score(y_test, knn_pred,average='macro')
knn_fscore = f1_score (y_test, knn_pred,average='macro')

print('knn_accuracy with N-GRAM = ',knn_accuracy)
print('knn_precision with N-GRAM = ',knn_precision)
print('knn_recall with N-GRAM = ',knn_recall)
print('knn_Fscore with N-GRAM = ',knn_fscore)
print("----------------------------------------------------------------")


#DECISION TREE 
# Train and evaluate the Decision Tree model
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
dt_pred = dt.predict(X_test)



dt_accuracy = accuracy_score(y_test, dt_pred)
dt_precision = precision_score(y_test, dt_pred,average='macro')
dt_recall = recall_score(y_test, dt_pred,average='macro')
dt_fscore = f1_score (y_test, dt_pred,average='macro')

print('dt_accuracy with N-GRAM  = ',dt_accuracy)
print('dt_precision with N-GRAM = ',dt_precision)
print('dt_recall with N-GRAM = ',dt_recall)
print('dt_Fscore with N-GRAM = ',dt_fscore)
print("----------------------------------------------------------------")



# Train and evaluate the SVM model
svm = SVC()
svm.fit(X_train, y_train)
svm_pred = svm.predict(X_test)


svm_accuracy = accuracy_score(y_test, svm_pred)
svm_precision = precision_score(y_test, svm_pred,average='macro')
svm_recall = recall_score(y_test, svm_pred,average='macro')
svm_fscore = f1_score (y_test, svm_pred,average='macro')


print('svm_accuracy with N-GRAM = ',svm_accuracy)
print('svm_precision with N-GRAM = ',svm_precision)
print('svm_recall with N-GRAM = ',svm_recall)
print('svm_Fscore with N-GRAM = ',svm_fscore)
print("----------------------------------------------------------------")

# Train and evaluate the XGBoost model
xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_train, y_train)
xgb_pred = xgb_model.predict(X_test)

xgb_accuracy = accuracy_score(y_test, xgb_pred)
xgb_precision = precision_score(y_test, xgb_pred,average='macro')
xgb_recall = recall_score(y_test, xgb_pred,average='macro')
xgb_fscore = f1_score (y_test, xgb_pred,average='macro')


print('xgb_accuracy with N-GRAM = ',xgb_accuracy)
print('xgb_precision with N-GRAM = ',xgb_precision)
print('xgb_recall with N-GRAM = ',xgb_recall)
print('xgb_Fscore with N-GRAM = ',xgb_fscore)
print("----------------------------------------------------------------")
#++++++++++++++++++++++++++++++++++++++++++++++++


# Plot the accuracy comparison of the models
models = ['KNN', 'SVM','DT', 'XGBoost']
accuracies = [knn_accuracy, svm_accuracy,dt_accuracy, xgb_accuracy]
plt.bar(models, accuracies)
plt.xlabel('Models')
plt.ylabel('Accuracy')
plt.title('Accuracy comparison of models')
plt.show()

# Plot the Precision comparison of the models
models = ['KNN', 'SVM','DT', 'XGBoost']
precisions = [knn_precision, svm_precision,dt_precision, xgb_precision]
plt.bar(models, precisions)
plt.xlabel('Models')
plt.ylabel('precision')
plt.title('Precision comparison of models')
plt.show()

# Plot the Recall comparison of the models
models = ['KNN', 'SVM','DT', 'XGBoost']
recalls = [knn_recall, svm_recall,dt_recall, xgb_recall]
plt.bar(models, recalls)
plt.xlabel('Models')
plt.ylabel('Recall')
plt.title('Recall comparison of models')
plt.show()

# Plot the Fscore comparison of the models
models = ['KNN', 'SVM','DT', 'XGBoost']
fscores = [knn_fscore, svm_fscore,dt_fscore, xgb_fscore]
plt.bar(models, fscores)
plt.xlabel('Models')
plt.ylabel('Fscore')
plt.title('Fscore comparison of models')
plt.show()

# CONFUSION MATRIX AND ERROR ANALYSIS FOR N-GRAM

In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix

# SVM ERROR ANALYSIS AND CONFUSION MATRIX

In [ ]:
temp=[]
for i in y_test.index:
  temp.append(data['Text'].iloc[i])
df=pd.DataFrame(temp,columns=['Text'])
svm_pred = svm.predict(X_test)
df['actual_data']=y_test.to_numpy()
df['predicted_data']=svm_pred
df

In [ ]:
df['Count']=df['predicted_data'] == df['actual_data']
count=0
for i in df['Count']:
  if i ==True:
    count=count+1
df.drop(labels=['Count'],axis=1,inplace=True)
print("The number of wrong predictions made by the SVM model with N-GRAM are:",200-count)

conf_matrix = confusion_matrix(y_test.to_numpy(), svm_pred)
conf_matrix_df = pd.DataFrame(conf_matrix, index=['Herman', 'Maria','Chesterton','Jane','Milton'], columns=['Herman', 'Maria','Chesterton','Jane','Milton'])
print('\nThe confusion matrix for the SVM model is \n\n',conf_matrix_df)
print("--------------------------------------------------------------------------------------------")

In [ ]:
sns.heatmap(conf_matrix_df, annot=True, fmt='d')
plt.title("Confusion Matrix for SVM Model")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

# KNN ERROR AND CONFUSION MATRIX

In [ ]:
temp=[]
for i in y_test.index:
  temp.append(data['Text'].iloc[i])
df=pd.DataFrame(temp,columns=['Text'])
knn_pred = knn.predict(X_test)
df['actual_data']=y_test.to_numpy()
df['predicted_data']=knn_pred
df

In [ ]:
df['Count']=df['predicted_data'] == df['actual_data']
count=0
for i in df['Count']:
  if i ==True:
    count=count+1
df.drop(labels=['Count'],axis=1,inplace=True)
print("The number of wrong predictions made by the KNN model with N-GRAM are:",200-count)

conf_matrix = confusion_matrix(y_test.to_numpy(), knn_pred)
conf_matrix_df = pd.DataFrame(conf_matrix, index=['Herman', 'Maria','Chesterton','Jane','Milton'], columns=['Herman', 'Maria','Chesterton','Jane','Milton'])
print('\nThe confusion matrix for the KNN model is \n\n',conf_matrix_df)
print("--------------------------------------------------------------------------------------------")

In [ ]:
sns.heatmap(conf_matrix_df, annot=True, fmt='d')
plt.title("Confusion Matrix for KNN Model")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

# Decision Tree ERROR AND CONFUSION MATRIX

In [ ]:
temp=[]
for i in y_test.index:
  temp.append(data['Text'].iloc[i])
df=pd.DataFrame(temp,columns=['Text'])
dt_pred = dt.predict(X_test)
df['actual_data']=y_test.to_numpy()
df['predicted_data']=dt_pred
df

In [ ]:
df['Count']=df['predicted_data'] == df['actual_data']
count=0
for i in df['Count']:
  if i ==True:
    count=count+1
df.drop(labels=['Count'],axis=1,inplace=True)
print("The number of wrong predictions made by the decision tree model with N-GRAM are:",200-count)

conf_matrix = confusion_matrix(y_test.to_numpy(), dt_pred)
conf_matrix_df = pd.DataFrame(conf_matrix, index=['Herman', 'Maria','Chesterton','Jane','Milton'], columns=['Herman', 'Maria','Chesterton','Jane','Milton'])
print('\nThe confusion matrix for the Decision Tree model is \n\n',conf_matrix_df)
print("--------------------------------------------------------------------------------------------")

In [ ]:
sns.heatmap(conf_matrix_df, annot=True, fmt='d')
plt.title("Confusion Matrix for Decision Tree Model")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

# XGBoost ERROR AND CONFUSION MATRIX

In [ ]:
temp=[]
for i in y_test.index:
  temp.append(data['Text'].iloc[i])
df=pd.DataFrame(temp,columns=['Text'])
xgb_pred = xgb_model.predict(X_test)
df['actual_data']=y_test.to_numpy()
df['predicted_data']=xgb_pred
df

In [ ]:
df['Count']=df['predicted_data'] == df['actual_data']
count=0
for i in df['Count']:
  if i ==True:
    count=count+1
df.drop(labels=['Count'],axis=1,inplace=True)
print("The number of wrong predictions made by the XGBoost model with N-GRAM are:",200-count)

conf_matrix = confusion_matrix(y_test.to_numpy(), xgb_pred)
conf_matrix_df = pd.DataFrame(conf_matrix, index=['Herman', 'Maria','Chesterton','Jane','Milton'], columns=['Herman', 'Maria','Chesterton','Jane','Milton'])
print('\nThe confusion matrix for the XGBoost model is \n\n',conf_matrix_df)
print("--------------------------------------------------------------------------------------------")

In [ ]:
sns.heatmap(conf_matrix_df, annot=True, fmt='d')
plt.title("Confusion Matrix for XGBoost Model")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

# 10 Fold cross validation

In [ ]:
print("10 fold validation for SVM \n")
accuracies=ten_fold_cross_validation(svm,ngrams,data['Author'])
print(accuracies)
print("Bias and Variability for SVM model is\n")
bias = 1-np.mean(accuracies)
variability = np.std(accuracies)

print("Bias: {:.3f}".format(bias))
print("Variability: {:.3f}".format(variability))

In [ ]:
print("10 fold validation for KNN \n")
accuracies=ten_fold_cross_validation(knn,ngrams,data['Author'])
print(accuracies)
print("Bias and Variability for KNN model is\n")
bias = 1-np.mean(accuracies)
variability = np.std(accuracies)

print("Bias: {:.3f}".format(bias))
print("Variability: {:.3f}".format(variability))

In [ ]:
print("10 fold validation for Decision Tree \n")
accuracies=ten_fold_cross_validation(dt,ngrams,data['Author'])
print(accuracies)
print("Bias and Variability for Decision Tree model is\n")
bias = 1-np.mean(accuracies)
variability = np.std(accuracies)

print("Bias: {:.3f}".format(bias))
print("Variability: {:.3f}".format(variability))

In [ ]:
print("10 fold validation for XGB \n")
accuracies=ten_fold_cross_validation(xgb_model,ngrams,data['Author'])
print(accuracies)
print("Bias and Variability for XGB model is\n")
bias = 1-np.mean(accuracies)
variability = np.std(accuracies)

print("Bias: {:.3f}".format(bias))
print("Variability: {:.3f}".format(variability))

10 fold validation for XGB 



# CHAMPION MODEL = SVM WITH TF-IDF AVERAGING AN ACCURACY OF 0.94

# 10 FOLD COSS VALIDATION FOR CHAMPION MODEL 


In [ ]:
import numpy as np
from sklearn.model_selection import KFold

def ten_fold_cross_validation(model, X, y):
    kf = KFold(n_splits=10)
    accuracy = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        model.fit(X_train, y_train)
        accuracy.append(model.score(X_test, y_test))
    
    return np.array(accuracy)


In [ ]:
print("10 fold validation for SVM ")
ten_fold_cross_validation(svm,tfidf,data['Author'])

In [ ]:
accuracy = ten_fold_cross_validation(svm,bag_of_words,data['Author'])

plt.bar(range(1, 11), accuracy, align='center')
plt.xticks(range(1, 11))
plt.xlabel('Fold Number')
plt.ylabel('Accuracy')
plt.title('10-Fold Cross Validation Accuracy')
plt.show()

BIAS AND VARIABILITY 

In [ ]:
bias = 1-np.mean(accuracies)
variability = np.std(accuracies)

print("Bias: {:.3f}".format(bias))
print("Variability: {:.3f}".format(variability))

# LOWERING THE ACCURACY 

the accuracy of the champion model has been lowered to 0.69 which is approx 20% 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# Read the csv file
#df = pd.read_csv('book_partitions.csv')

# Convert the text data into a tf-idf representation
vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(data['Text'].values)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(tfidf, data['Author'], test_size=0.8, random_state=42)

# K-NEAREST NEIGHBOUR 
# Train and evaluate the KNN model
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_test)


knn_accuracy = accuracy_score(y_test, knn_pred)
knn_precision = precision_score(y_test, knn_pred,average='micro')
knn_recall = recall_score(y_test, knn_pred,average='micro')
knn_fscore = f1_score (y_test, knn_pred,average='micro')

print('knn_accuracy with TF-IDF = ',knn_accuracy)
print('knn_precision with TF-IDF = ',knn_precision)
print('knn_recall with TF-IDF = ',knn_recall)
print('knn_Fscore with TF-IDF = ',knn_fscore)
print("----------------------------------------------------------------")


#DECISION TREE 
# Train and evaluate the Decision Tree model
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
dt_pred = dt.predict(X_test)



dt_accuracy = accuracy_score(y_test, dt_pred)
dt_precision = precision_score(y_test, dt_pred,average='micro')
dt_recall = recall_score(y_test, dt_pred,average='micro')
dt_fscore = f1_score (y_test, dt_pred,average='micro')

print('dt_accuracy with TF-IDF  = ',dt_accuracy)
print('dt_precision with TF-IDF = ',dt_precision)
print('dt_recall with TF-IDF = ',dt_recall)
print('dt_Fscore with TF-IDF = ',dt_fscore)
print("----------------------------------------------------------------")



# Train and evaluate the SVM model
svm = SVC()
svm.fit(X_train, y_train)
svm_pred = svm.predict(X_test)


svm_accuracy = accuracy_score(y_test, svm_pred)
svm_precision = precision_score(y_test, svm_pred,average='micro')
svm_recall = recall_score(y_test, svm_pred,average='micro')
svm_fscore = f1_score (y_test, svm_pred,average='micro')


print('svm_accuracy with TF-IDF = ',svm_accuracy)
print('svm_precision with TF-IDF = ',svm_precision)
print('svm_recall with TF-IDF = ',svm_recall)
print('svm_Fscore with TF-IDF = ',svm_fscore)
print("----------------------------------------------------------------")

# Train and evaluate the XGBoost model
xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_train, y_train)
xgb_pred = xgb_model.predict(X_test)

xgb_accuracy = accuracy_score(y_test, xgb_pred)
xgb_precision = precision_score(y_test, xgb_pred,average='micro')
xgb_recall = recall_score(y_test, xgb_pred,average='micro')
xgb_fscore = f1_score (y_test, xgb_pred,average='micro')


print('xgb_accuracy with TF-IDF = ',xgb_accuracy)
print('xgb_precision with TF-IDF = ',xgb_precision)
print('xgb_recall with TF-IDF = ',xgb_recall)
print('xgb_Fscore with TF-IDF = ',xgb_fscore)
print("----------------------------------------------------------------")
#++++++++++++++++++++++++++++++++++++++++++++++++


# Plot the accuracy comparison of the models
models = ['KNN', 'SVM','DT', 'XGBoost']
accuracies = [knn_accuracy, svm_accuracy,dt_accuracy, xgb_accuracy]
plt.bar(models, accuracies)
plt.xlabel('Models')
plt.ylabel('Accuracy')
plt.title('Accuracy comparison of models')
plt.show()

# Plot the Precision comparison of the models
models = ['KNN', 'SVM','DT', 'XGBoost']
precisions = [knn_precision, svm_precision,dt_precision, xgb_precision]
plt.bar(models, precisions)
plt.xlabel('Models')
plt.ylabel('precision')
plt.title('Precision comparison of models')
plt.show()

# Plot the Recall comparison of the models
models = ['KNN', 'SVM','DT', 'XGBoost']
recalls = [knn_recall, svm_recall,dt_recall, xgb_recall]
plt.bar(models, recalls)
plt.xlabel('Models')
plt.ylabel('Recall')
plt.title('Recall comparison of models')
plt.show()

# Plot the Fscore comparison of the models
models = ['KNN', 'SVM','DT', 'XGBoost']
fscores = [knn_fscore, svm_fscore,dt_fscore, xgb_fscore]
plt.bar(models, fscores)
plt.xlabel('Models')
plt.ylabel('Fscore')
plt.title('Fscore comparison of models')
plt.show()